In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from nicegui import ui 
import joblib
import warnings
import asyncio
from ipywidgets import widgets, VBox, Button, Output
from flask import Flask, request, jsonify
import gradio as gr
from dash import Dash, html, dcc, Input, Output
warnings.filterwarnings("ignore")

from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.metrics import accuracy_score, classification_report, roc_curve
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [2]:
df = pd.read_csv("heart.csv")

In [3]:
dft = df.copy()

In [4]:
df.shape

(303, 14)

In [5]:
df.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [6]:
df.isnull().sum()

age         0
sex         0
cp          0
trtbps      0
chol        0
fbs         0
restecg     0
thalachh    0
exng        0
oldpeak     0
slp         0
caa         0
thall       0
output      0
dtype: int64

In [7]:
X = df.drop(['output'],axis=1)
y = df[['output']]

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 42)

In [9]:
logreg = LogisticRegression()

logreg.fit(X_train, y_train)

y_pred_proba = logreg.predict_proba(X_test)

y_pred = np.argmax(y_pred_proba,axis=1)

print("The test accuracy score of Logistric Regression is ", accuracy_score(y_test, y_pred))

The test accuracy score of Logistric Regression is  0.8852459016393442


In [11]:
joblib.dump(logreg, 'logreg_model.pkl')

['logreg_model.pkl']

In [12]:
loaded_model = joblib.load('logreg_model.pkl')

In [12]:
# Modeli yükleyin
model = joblib.load('logreg_model.pkl')

# Dash uygulamasını başlatın
app = Dash(__name__)

# Uygulama tasarımı
app.layout = html.Div([
    html.H1("Tahmin Uygulaması", style={'textAlign': 'center', 'marginBottom': '20px'}),
    html.Div([
        html.Div([
            html.Label("Yaş:", style={'display': 'block'}),
            dcc.Input(id='input-age', type='number', placeholder='Yaş', style={'marginBottom': '10px', 'width': '100%'})
        ]),
        
        html.Div([
            html.Label("Cinsiyet (0: Kadın, 1: Erkek):", style={'display': 'block'}),
            dcc.Input(id='input-sex', type='number', placeholder='Cinsiyet', style={'marginBottom': '10px', 'width': '100%'})
        ]),

        html.Div([
            html.Label("Göğüs Ağrısı Tipi (0-3):", style={'display': 'block'}),
            dcc.Input(id='input-cp', type='number', placeholder='Göğüs Ağrısı Tipi', style={'marginBottom': '10px', 'width': '100%'})
        ]),

        html.Div([
            html.Label("Dinlenme Kan Basıncı:", style={'display': 'block'}),
            dcc.Input(id='input-trestbps', type='number', placeholder='Dinlenme Kan Basıncı', style={'marginBottom': '10px', 'width': '100%'})
        ]),

        html.Div([
            html.Label("Kolesterol Seviyesi:", style={'display': 'block'}),
            dcc.Input(id='input-chol', type='number', placeholder='Kolesterol Seviyesi', style={'marginBottom': '10px', 'width': '100%'})
        ]),

        html.Div([
            html.Label("Açlık Kan Şekeri (>120mg/dl, 1: True, 0: False):", style={'display': 'block'}),
            dcc.Input(id='input-fbs', type='number', placeholder='Açlık Kan Şekeri', style={'marginBottom': '10px', 'width': '100%'})
        ]),

        html.Div([
            html.Label("Rest ECG (0-2):", style={'display': 'block'}),
            dcc.Input(id='input-restecg', type='number', placeholder='Rest ECG', style={'marginBottom': '10px', 'width': '100%'})
        ]),

        html.Div([
            html.Label("Maksimum Kalp Atım Hızı:", style={'display': 'block'}),
            dcc.Input(id='input-thalach', type='number', placeholder='Maksimum Kalp Atım Hızı', style={'marginBottom': '10px', 'width': '100%'})
        ]),

        html.Div([
            html.Label("Egzersiz ile Ağrı (1: Evet, 0: Hayır):", style={'display': 'block'}),
            dcc.Input(id='input-exang', type='number', placeholder='Egzersiz ile Ağrı', style={'marginBottom': '10px', 'width': '100%'})
        ]),

        html.Div([
            html.Label("ST Depresyonu:", style={'display': 'block'}),
            dcc.Input(id='input-oldpeak', type='number', placeholder='ST Depresyonu', style={'marginBottom': '10px', 'width': '100%'})
        ]),

        html.Div([
            html.Label("Eğim (0-2):", style={'display': 'block'}),
            dcc.Input(id='input-slope', type='number', placeholder='Eğim', style={'marginBottom': '10px', 'width': '100%'})
        ]),

        html.Div([
            html.Label("Renkli Damar Sayısı (0-4):", style={'display': 'block'}),
            dcc.Input(id='input-ca', type='number', placeholder='Renkli Damar Sayısı', style={'marginBottom': '10px', 'width': '100%'})
        ]),

        html.Div([
            html.Label("THAL (1: Normal, 2: Sabit Hata, 3: Ters Hata):", style={'display': 'block'}),
            dcc.Input(id='input-thal', type='number', placeholder='THAL', style={'marginBottom': '10px', 'width': '100%'})
        ]),
    ], style={'maxWidth': '400px', 'margin': '0 auto'}),
    
    html.Button('Tahmin Et', id='predict-btn', n_clicks=0, style={
        'display': 'block', 'margin': '20px auto', 'padding': '10px 20px', 'fontSize': '16px'
    }),
    html.Div(id='output', style={'marginTop': '20px', 'textAlign': 'center', 'fontSize': '20px'})
])

# Callback fonksiyonu
@app.callback(
    Output('output', 'children'),
    Input('predict-btn', 'n_clicks'),
    [Input(f'input-{field}', 'value') for field in [
        'age', 'sex', 'cp', 'trestbps', 'chol', 'fbs',
        'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal']]
)
def predict(n_clicks, *inputs):
    if n_clicks > 0:
        if None in inputs:
            return "Lütfen tüm alanları doldurun!"
        try:
            new_data = np.array([inputs])
            prediction = model.predict(new_data)
            probabilities = model.predict_proba(new_data)
            risk_percentage = probabilities[0][1] * 100  # 1. sınıf (kalp krizi) olasılığı
            return f"Tahmin Sonucu: {'Pozitif' if prediction[0] == 1 else 'Negatif'}\n" \
                   f"Kalp Krizi Geçirme Riski: %{risk_percentage:.2f}"
        except Exception as e:
            return f"Bir hata oluştu: {str(e)}"
    return ""

if __name__ == '__main__':
    app.run_server(debug=True,port=8040)


# Modeli yükleyin
model = joblib.load('logreg_model.pkl')

# Dash uygulamasını başlatın
app = Dash(__name__)

# Uygulama tasarımı
app.layout = html.Div([
    html.H1("Tahmin Uygulaması", style={'textAlign': 'center', 'marginBottom': '20px'}),
    html.Div([
        dcc.Input(id='input-age', type='number', placeholder='Yaş', style={'marginBottom': '10px', 'width': '100%'}),
        dcc.Input(id='input-sex', type='number', placeholder='Cinsiyet (0: Kadın, 1: Erkek)', style={'marginBottom': '10px', 'width': '100%'}),
        dcc.Input(id='input-cp', type='number', placeholder='Göğüs Ağrısı Tipi (0-3)', style={'marginBottom': '10px', 'width': '100%'}),
        dcc.Input(id='input-trestbps', type='number', placeholder='Dinlenme Kan Basıncı', style={'marginBottom': '10px', 'width': '100%'}),
        dcc.Input(id='input-chol', type='number', placeholder='Kolesterol Seviyesi', style={'marginBottom': '10px', 'width': '100%'}),
        dcc.Input(id='input-fbs', type='number', placeholder='Açlık Kan Şekeri (>120mg/dl, 1: True, 0: False)', style={'marginBottom': '10px', 'width': '100%'}),
        dcc.Input(id='input-restecg', type='number', placeholder='Rest ECG (0-2)', style={'marginBottom': '10px', 'width': '100%'}),
        dcc.Input(id='input-thalach', type='number', placeholder='Maksimum Kalp Atım Hızı', style={'marginBottom': '10px', 'width': '100%'}),
        dcc.Input(id='input-exang', type='number', placeholder='Egzersiz ile Ağrı (1: Evet, 0: Hayır)', style={'marginBottom': '10px', 'width': '100%'}),
        dcc.Input(id='input-oldpeak', type='number', placeholder='ST Depresyonu', style={'marginBottom': '10px', 'width': '100%'}),
        dcc.Input(id='input-slope', type='number', placeholder='Eğim (0-2)', style={'marginBottom': '10px', 'width': '100%'}),
        dcc.Input(id='input-ca', type='number', placeholder='Renkli Damar Sayısı (0-4)', style={'marginBottom': '10px', 'width': '100%'}),
        dcc.Input(id='input-thal', type='number', placeholder='THAL (1: Normal, 2: Sabit Hata, 3: Ters Hata)', style={'marginBottom': '10px', 'width': '100%'}),
    ], style={'maxWidth': '400px', 'margin': '0 auto'}),
    html.Button('Tahmin Et', id='predict-btn', n_clicks=0, style={
        'display': 'block', 'margin': '20px auto', 'padding': '10px 20px', 'fontSize': '16px'
    }),
    html.Div(id='output', style={'marginTop': '20px', 'textAlign': 'center', 'fontSize': '20px'})
])

# Callback fonksiyonu
@app.callback(
    Output('output', 'children'),
    Input('predict-btn', 'n_clicks'),
    [Input(f'input-{field}', 'value') for field in [
        'age', 'sex', 'cp', 'trestbps', 'chol', 'fbs',
        'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal']]
)
def predict(n_clicks, *inputs):
    if n_clicks > 0:
        if None in inputs:
            return "Lütfen tüm alanları doldurun!"
        try:
            new_data = np.array([inputs])
            prediction = model.predict(new_data)
            return f"Tahmin Sonucu: {'Pozitif' if prediction[0] == 1 else 'Negatif'}"
        except Exception as e:
            return f"Bir hata oluştu: {str(e)}"
    return ""

if __name__ == '__main__':
    app.run_server(debug=True)


Decision Tree

In [9]:
dt = DecisionTreeClassifier(random_state = 42)

# fitting the model
dt.fit(X_train, y_train)

# calculating the predictions
y_pred = dt.predict(X_test)

# printing the test accuracy
print("The test accuracy score of Decision Tree is ", accuracy_score(y_test, y_pred))

The test accuracy score of Decision Tree is  0.7540983606557377


Log Reg

Random Forest

In [11]:
rf = RandomForestClassifier()

rf.fit(X_train, y_train)

y_pred = dt.predict(X_test)

print("The test accuracy score of Random Forest is ", accuracy_score(y_test, y_pred))

The test accuracy score of Random Forest is  0.7540983606557377


GBT

In [14]:
gbt = GradientBoostingClassifier(n_estimators = 300,max_depth=1,subsample=0.8,max_features=0.2,random_state=42)

gbt.fit(X_train,y_train)

y_pred = gbt.predict(X_test)
print("The test accuracy score of Gradient Boosting Classifier is ", accuracy_score(y_test, y_pred))

The test accuracy score of Gradient Boosting Classifier is  0.8688524590163934


SVM

In [13]:
clf = SVC(kernel='linear', C=1, random_state=42).fit(X_train,y_train)

y_pred = clf.predict(X_test)

print("The test accuracy score of SVM is ", accuracy_score(y_test, y_pred))

The test accuracy score of SVM is  0.8688524590163934


In [14]:
svm = SVC()

parameters = {"C":np.arange(1,10,1),'gamma':[0.00001,0.00005, 0.0001,0.0005,0.001,0.005,0.01,0.05,0.1,0.5,1,5]}

searcher = GridSearchCV(svm, parameters)

searcher.fit(X_train, y_train)

print("The best params are :", searcher.best_params_)
print("The best score is   :", searcher.best_score_)

y_pred = searcher.predict(X_test)

print("The test accuracy score of SVM after hyper-parameter tuning is ", accuracy_score(y_test, y_pred))

The best params are : {'C': 5, 'gamma': 0.0005}
The best score is   : 0.6855442176870749
The test accuracy score of SVM after hyper-parameter tuning is  0.7049180327868853


In [17]:
new_data = np.array([[63, 1, 3, 145, 233, 1, 0, 150, 0, 2.3, 0, 0, 1]])
prediction = loaded_model.predict(new_data)
print("Tahmin sonucu: ", prediction)

Tahmin sonucu:  [1]


In [17]:
loaded_model_gbt = joblib.load('gbt_model.pkl')

In [16]:
joblib.dump(gbt, 'gbt_model.pkl')

['gbt_model.pkl']

In [20]:
prediction = loaded_model_gbt.predict(new_data)
print("Tahmin sonucu: ", prediction)

Tahmin sonucu:  [1]


In [19]:
# Modeli yükleyin
model = joblib.load('gbt_model.pkl')

# Dash uygulamasını başlatın
app = Dash(__name__)

# Uygulama tasarımı
app.layout = html.Div([
    html.H1("Tahmin Uygulaması", style={'textAlign': 'center', 'marginBottom': '20px'}),
    html.Div([
        html.Div([
            html.Label("Yaş:", style={'display': 'block'}),
            dcc.Input(id='input-age', type='number', placeholder='Yaş', style={'marginBottom': '10px', 'width': '100%'})
        ]),
        
        html.Div([
            html.Label("Cinsiyet (0: Kadın, 1: Erkek):", style={'display': 'block'}),
            dcc.Input(id='input-sex', type='number', placeholder='Cinsiyet', style={'marginBottom': '10px', 'width': '100%'})
        ]),

        html.Div([
            html.Label("Göğüs Ağrısı Tipi (0-3):", style={'display': 'block'}),
            dcc.Input(id='input-cp', type='number', placeholder='Göğüs Ağrısı Tipi', style={'marginBottom': '10px', 'width': '100%'})
        ]),

        html.Div([
            html.Label("Dinlenme Kan Basıncı:", style={'display': 'block'}),
            dcc.Input(id='input-trestbps', type='number', placeholder='Dinlenme Kan Basıncı', style={'marginBottom': '10px', 'width': '100%'})
        ]),

        html.Div([
            html.Label("Kolesterol Seviyesi:", style={'display': 'block'}),
            dcc.Input(id='input-chol', type='number', placeholder='Kolesterol Seviyesi', style={'marginBottom': '10px', 'width': '100%'})
        ]),

        html.Div([
            html.Label("Açlık Kan Şekeri (>120mg/dl, 1: True, 0: False):", style={'display': 'block'}),
            dcc.Input(id='input-fbs', type='number', placeholder='Açlık Kan Şekeri', style={'marginBottom': '10px', 'width': '100%'})
        ]),

        html.Div([
            html.Label("Rest ECG (0-2):", style={'display': 'block'}),
            dcc.Input(id='input-restecg', type='number', placeholder='Rest ECG', style={'marginBottom': '10px', 'width': '100%'})
        ]),

        html.Div([
            html.Label("Maksimum Kalp Atım Hızı:", style={'display': 'block'}),
            dcc.Input(id='input-thalach', type='number', placeholder='Maksimum Kalp Atım Hızı', style={'marginBottom': '10px', 'width': '100%'})
        ]),

        html.Div([
            html.Label("Egzersiz ile Ağrı (1: Evet, 0: Hayır):", style={'display': 'block'}),
            dcc.Input(id='input-exang', type='number', placeholder='Egzersiz ile Ağrı', style={'marginBottom': '10px', 'width': '100%'})
        ]),

        html.Div([
            html.Label("ST Depresyonu:", style={'display': 'block'}),
            dcc.Input(id='input-oldpeak', type='number', placeholder='ST Depresyonu', style={'marginBottom': '10px', 'width': '100%'})
        ]),

        html.Div([
            html.Label("Eğim (0-2):", style={'display': 'block'}),
            dcc.Input(id='input-slope', type='number', placeholder='Eğim', style={'marginBottom': '10px', 'width': '100%'})
        ]),

        html.Div([
            html.Label("Renkli Damar Sayısı (0-4):", style={'display': 'block'}),
            dcc.Input(id='input-ca', type='number', placeholder='Renkli Damar Sayısı', style={'marginBottom': '10px', 'width': '100%'})
        ]),

        html.Div([
            html.Label("THAL (1: Normal, 2: Sabit Hata, 3: Ters Hata):", style={'display': 'block'}),
            dcc.Input(id='input-thal', type='number', placeholder='THAL', style={'marginBottom': '10px', 'width': '100%'})
        ]),
    ], style={'maxWidth': '400px', 'margin': '0 auto'}),
    
    html.Button('Tahmin Et', id='predict-btn', n_clicks=0, style={
        'display': 'block', 'margin': '20px auto', 'padding': '10px 20px', 'fontSize': '16px'
    }),
    html.Div(id='output', style={'marginTop': '20px', 'textAlign': 'center', 'fontSize': '20px'})
])

# Callback fonksiyonu
@app.callback(
    Output('output', 'children'),
    Input('predict-btn', 'n_clicks'),
    [Input(f'input-{field}', 'value') for field in [
        'age', 'sex', 'cp', 'trestbps', 'chol', 'fbs',
        'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal']]
)
def predict(n_clicks, *inputs):
    if n_clicks > 0:
        if None in inputs:
            return "Lütfen tüm alanları doldurun!"
        try:
            new_data = np.array([inputs])
            prediction = model.predict(new_data)
            probabilities = model.predict_proba(new_data)
            risk_percentage = probabilities[0][1] * 100  # 1. sınıf (kalp krizi) olasılığı
            return f"Tahmin Sonucu: {'Pozitif' if prediction[0] == 1 else 'Negatif'}\n" \
                   f"Kalp Krizi Geçirme Riski: %{risk_percentage:.2f}"
        except Exception as e:
            return f"Bir hata oluştu: {str(e)}"
    return ""

if __name__ == '__main__':
    app.run_server(debug=True,port=8030)
